In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
sys.path.append(os.path.abspath("databases/"))
import dwh
import numpy as np
import pandas as pd
from multiprocessing import Pool, cpu_count
import pickle
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, classification_report
from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool, cpu_count
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

/home/vscode/.local/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Будем использовать данные подготовленные в "hog_model.ipynb" и сохранённые в pickle

In [3]:
%%time
# Prepared data
try:
    pickle_obj
except NameError:
    with open('hog_model', 'rb') as file:
        pickle_obj = pickle.load(file)

# Pickle with ready to use models
try:
    pckl_models
except NameError:
    with open('models.pickle', 'rb') as file:
        pckl_models = pickle.load(file)
        
print(pickle_obj.keys())
print(pckl_models.keys())


dict_keys(['one_hot_enc', 'lreg', 'features_train', 'target_train', 'target_val', 'features_val'])
dict_keys(['lreg', 'sgd', 'rnd_forest', 'lgbm', 'lgbm2'])
CPU times: user 1.03 s, sys: 14.2 s, total: 15.2 s
Wall time: 15 s


In [4]:
def make_pred(estimator, 
              x_train=pickle_obj['features_train'], 
              x_val=pickle_obj['features_val']):
    ''' 
    Function makes prediction with prepared estimator.
    It returns predictions for train and validation datasets.
    '''
    return {'train': estimator.predict(np.array(x_train)), 'val': estimator.predict(np.array(x_val))}
    
def show_scores(predictions, 
                y_train=pickle_obj['target_train'],
                y_val=pickle_obj['target_val']):
    '''
    Func shows scores for train and val
    '''
    print('Train score:')
    print(f1_score(y_train, predictions['train'], average='micro'))
    print('Val scores:')
    print(f1_score(y_val, predictions['val'], average='micro'))
    print(classification_report(y_val, predictions['val'], digits=3))
    pass

In [7]:
%%time
pckl_models['lreg'] = LogisticRegression(random_state=42, 
                                         warm_start=True, 
                                         n_jobs=cpu_count()-1)
pckl_models['lreg'].fit(np.array(pickle_obj['features_train']), np.array(pickle_obj['target_train']))


CPU times: user 15.2 s, sys: 21.3 s, total: 36.5 s
Wall time: 12min 18s


/home/vscode/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(n_jobs=7, random_state=42, warm_start=True)

In [8]:
show_scores(make_pred(pckl_models['lreg']))

Train score:
0.769879809242729
Val scores:
0.7610739195584446
               precision    recall  f1-score   support

         bike      0.466     0.054     0.096      1007
          bus      0.555     0.098     0.166      1597
          car      0.802     0.902     0.849    102506
        motor      0.000     0.000     0.000       452
       person      0.634     0.523     0.573     13262
        rider      0.000     0.000     0.000       649
traffic light      0.700     0.653     0.676     26885
 traffic sign      0.721     0.662     0.690     34908
        train      0.000     0.000     0.000        15
        truck      0.510     0.220     0.308      4245

     accuracy                          0.761    185526
    macro avg      0.439     0.311     0.336    185526
 weighted avg      0.745     0.761     0.747    185526



In [78]:
%%time
pckl_models['sgd'] = SGDClassifier(random_state=42, 
                                   warm_start=True, 
                                   n_jobs=cpu_count()-1)
pckl_models['sgd'].fit(np.array(pickle_obj['features_train']), np.array(pickle_obj['target_train']))

SGDClassifier(n_jobs=7, random_state=42, warm_start=True)

In [15]:
show_scores(make_pred(pckl_models['sgd']))

Train score:
0.7639398199197901
Val scores:
0.7557431303429168
               precision    recall  f1-score   support

         bike      0.500     0.002     0.004      1007
          bus      0.544     0.019     0.037      1597
          car      0.783     0.919     0.846    102506
        motor      0.000     0.000     0.000       452
       person      0.666     0.450     0.537     13262
        rider      0.000     0.000     0.000       649
traffic light      0.705     0.641     0.671     26885
 traffic sign      0.721     0.650     0.684     34908
        train      0.000     0.000     0.000        15
        truck      0.345     0.023     0.042      4245

     accuracy                          0.756    185526
    macro avg      0.426     0.270     0.282    185526
 weighted avg      0.733     0.756     0.733    185526



In [84]:
%%time
pckl_models['rnd_forest'] = RandomForestClassifier(random_state=42,
                                                   warm_start=True, 
                                                   n_jobs=cpu_count()-1)
pckl_models['rnd_forest'].fit(np.array(pickle_obj['features_train']), np.array(pickle_obj['target_train']))

CPU times: user 4h 16min 40s, sys: 13.9 s, total: 4h 16min 54s
Wall time: 38min 15s


RandomForestClassifier(n_jobs=7, random_state=42, warm_start=True)

In [16]:
show_scores(make_pred(pckl_models['rnd_forest']))

Train score:
0.9999712480893578
Val scores:
0.7194301607321885
               precision    recall  f1-score   support

         bike      0.000     0.000     0.000      1007
          bus      0.889     0.005     0.010      1597
          car      0.710     0.955     0.815    102506
        motor      0.000     0.000     0.000       452
       person      0.832     0.056     0.104     13262
        rider      0.000     0.000     0.000       649
traffic light      0.784     0.498     0.609     26885
 traffic sign      0.721     0.611     0.661     34908
        train      0.000     0.000     0.000        15
        truck      0.808     0.028     0.054      4245

     accuracy                          0.719    185526
    macro avg      0.474     0.215     0.225    185526
 weighted avg      0.727     0.719     0.672    185526



In [21]:
%%time
pckl_models['lgbm'] = LGBMClassifier(objective='multiclassova', 
                                     random_state=42,
                                     n_jobs=7,
                                     n_estimators=100,
                                     max_depth=6)
pckl_models['lgbm'].fit(np.array(pickle_obj['features_train']), np.array(pickle_obj['target_train']))

CPU times: user 7h 1min 38s, sys: 8.86 s, total: 7h 1min 47s
Wall time: 1h 1min 38s


LGBMClassifier(max_depth=6, n_jobs=7, objective='multiclassova',
               random_state=42)

In [18]:
show_scores(make_pred(pckl_models['lgbm']))

Train score:
0.7825120000372997
Val scores:
0.7680001724825631
               precision    recall  f1-score   support

         bike      0.432     0.048     0.086      1007
          bus      0.609     0.084     0.147      1597
          car      0.780     0.934     0.850    102506
        motor      0.014     0.007     0.009       452
       person      0.725     0.409     0.523     13262
        rider      0.042     0.008     0.013       649
traffic light      0.771     0.635     0.697     26885
 traffic sign      0.746     0.671     0.706     34908
        train      0.000     0.000     0.000        15
        truck      0.671     0.141     0.233      4245

     accuracy                          0.768    185526
    macro avg      0.479     0.294     0.326    185526
 weighted avg      0.758     0.768     0.748    185526



In [26]:
%%time
pckl_models['lgbm2'] = LGBMClassifier(objective='multiclassova', 
                                      random_state=42,
                                      n_jobs=7,
                                      n_estimators=100,
                                      max_depth=8)
pckl_models['lgbm2'].fit(np.array(pickle_obj['features_train']), np.array(pickle_obj['target_train']))

CPU times: user 6h 59min 51s, sys: 9.01 s, total: 7h
Wall time: 1h 1min 19s


LGBMClassifier(max_depth=8, n_jobs=7, objective='multiclassova',
               random_state=42)

In [6]:
show_scores(make_pred(pckl_models['lgbm2']))

Train score:
0.7853102603135823
Val scores:
0.7690350678611085
               precision    recall  f1-score   support

         bike      0.276     0.051     0.086      1007
          bus      0.571     0.098     0.167      1597
          car      0.783     0.932     0.851    102506
        motor      0.006     0.004     0.005       452
       person      0.719     0.418     0.528     13262
        rider      0.073     0.025     0.037       649
traffic light      0.774     0.639     0.700     26885
 traffic sign      0.750     0.673     0.709     34908
        train      0.004     0.067     0.007        15
        truck      0.660     0.156     0.253      4245

     accuracy                          0.769    185526
    macro avg      0.462     0.306     0.334    185526
 weighted avg      0.759     0.769     0.751    185526



In [10]:
with open('models.pickle', 'wb') as file:
    pickle.dump(pckl_models, file)
    
with open('best_hog_estimator.pickle', 'wb') as file:
    pickle.dump(pckl_models['lgbm2'], file)